# Font recognition - improved models


## Load data and train-validation split

**Data is loaded, observed and treated**

In [180]:
import numpy as np
import pandas as pd

train_data = pd.read_csv('data/train_data.csv')
train_labels = pd.read_csv('data/train_labels.csv')


Labels are factorized and a full dataframe is constructed adding the encoded values as the last column

In [185]:
label_encoded, unique_labels = pd.factorize(train_labels['Font'])
labels = pd.DataFrame(label_encoded, columns=['label'])
df = pd.concat([train_data, labels], axis = 1)

**Train and validation split is conducted**

In [189]:
from sklearn.model_selection import train_test_split
X = df.iloc[:, :-1]
Y = df.iloc[:, -1]
x_train_df, x_valid_df, y_train_df, y_valid_df = train_test_split(X, Y, test_size=0.3, random_state = 0)

**Finally, the test data is loaded as well**

In [193]:
test_data = pd.read_csv('data/test_data.csv')

## Normalization of data

Now df has all the needed information. It will be transformed to a np.array for easier treatment within sklearn package

In [196]:
x_train_pre_norm = np.array(x_train_df)
x_valid_pre_norm = np.array(x_valid_df)
y_train = np.array(y_train_df)
y_valid = np.array(y_valid_df)

In [197]:
x_test_pre_norm = np.array(x_test_df)

In [198]:
X_np = np.array(X)

`mean` and `std` are obtained from full dataset

In [199]:
mean = np.sum(X_np, axis = 0) / X_np.shape[0]
std = np.std(X_np, axis = 0)

Implement normalization function from Homework 9

In [200]:
def normalize(X, mean, std):
    """Normalizes a given array X by columns 
    with the mean and std"""
    X_out = np.zeros(X.shape)
    X_out = (X - mean)/std
    return X_out 

In [201]:
x_train = normalize(x_train_pre_norm, mean, std)
x_valid = normalize(x_valid_pre_norm, mean, std)

In [202]:
x_test = normalize(x_test_pre_norm, mean, std)

Check normalization has been done correctly:
$$\text{mean}=0$$ $$\text{std}= 1$$

In [203]:
np.mean(x_train, axis = 0)[0:4]

array([ 0.00081244, -0.00083289, -0.00177084, -0.00353234])

In [204]:
np.std(x_valid, axis = 0)[0:4]

array([1.0004818 , 1.00024253, 1.00063935, 0.98696497])

In [205]:
np.mean(x_test, axis = 0)[0:4]

array([ 0.00514263, -0.00069739,  0.00495033, -0.00141616])

In [206]:
np.std(x_test, axis = 0)[0:4]

array([1.00905403, 0.99991203, 1.00076391, 1.00143199])

This allows us to have two different sets of features: with and without normalization. The normalization set will be used

## Function to save submission csv

A function will be created that saves predictions as a csv with the correct format

In [207]:
def predictions_as_csv(y_pred, file_name):
    path = "submissions/"
    status = 0
    if len(y_pred) == 29221:
        ids = np.arange(1,len(y_pred)+1,1)
        pred_label = unique_labels[y_pred]
        data = {'ID':ids, 'Font':pred_label} 
        submission = pd.DataFrame(data)
        submission.to_csv(path + file_name + ".csv", index = False)
        status = 1
    
    return status